# let's make a datframe split a couple ways by age, then see how harmonization with neurocombat changes it



## import libraries

In [ ]:
import os

import pandas as pd
import numpy as np


## prepare data

In [ ]:
filepath_mri = '../open_work/internal_results/cleaned_pvc2s/' 
filename_mri = os.path.join(filepath_mri,'StrokeMRI_pvc2c.csv') 

In [ ]:
StrokeMRI  = pd.read_csv(filename_mri)
StrokeMRI = StrokeMRI.drop(StrokeMRI.columns[0],axis=1)
sex_mapping = {'F':0,'M':1}
StrokeMRI = StrokeMRI.assign(sex = StrokeMRI.sex.map(sex_mapping))
StrokeMRI.head(3)

In [ ]:
StrokeMRI.age.describe()

In [ ]:
def split_frame_half_balanced_by_column(frame, column):
    """This is useful in cases where you want to split on a columns
    with continous values e.g. age."""
    df = frame.sort_values(column).reset_index()
    rng_even = range(0, len(df),2)
    rng_odd =  range(1, len(df),2)
    even_rows = df.iloc[rng_even]
    odd_rows = df.iloc[rng_odd]
    return even_rows, odd_rows

In [ ]:
stroke_even_unharmonized = split_frame_half_balanced_by_column(StrokeMRI, 'age')[0]
stroke_odd_unharmonized = split_frame_half_balanced_by_column(StrokeMRI, 'age')[1]

In [ ]:
def top_and_bottom_by_column(frame, column):
    """This is useful in cases where you want to split on a columns
    with continous values e.g. age.; and upi
    want the highest and lowest values seperated"""
    df = frame.sort_values(column).reset_index()
    len_first_half= len(df) // 2
    top = df.iloc[:len_first_half]
    bottom = df.iloc[len_first_half :]
    return top, bottom

In [ ]:
stroke_top_unharmonized = top_and_bottom_by_column(StrokeMRI, 'age')[0]
stroke_bottom_unharmonized = top_and_bottom_by_column(StrokeMRI, 'age')[1]